# Component Demonstration Notebook

A notebook to demonstrate some of the features of the Component class

Author: gavin.treseder@essentialenergy.com.au

In [1]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np 
import pandas as pd

import plotly.express as px

from pof.component import Component

In [2]:
comp = Component().set_demo()

In [3]:
comp.mc_timeline(t_end=100, n_iterations=1000)

100%|██████████| 1000/1000 [00:12<00:00, 82.10it/s]


In [4]:
cdf = comp.expected_cdf(t_end=1000)

In [5]:
erc = comp.expected_risk_cost()

In [11]:
df = comp.expected_risk_cost_df()

In [20]:
fig = px.area(
    df.reset_index(),
    x="time",
    y="cumulative_cost",
    color="task",
    line_group = 'failure_mode'
)

In [23]:
fig

In [32]:
n_time = 1001
time = list(np.linspace(0,n_time - 1, n_time))

failure_modes = []
tasks = []
times = []
costs = []
cumulative_costs = []

n_fm = len(erc)

for fm, fm_values in erc.items():

    for task, task_values in fm_values.items():
        failure_mode = np.full(n_time, fm)
        task = np.full(n_time, task)
        cost = np.full(n_time, 0)
        cost[task_values['time']] = task_values['cost']
        cumulative_cost = cost.cumsum()

        failure_modes = np.append(failure_modes, failure_mode)
        tasks = np.append(tasks, task)
        costs = np.append(costs, cost)
        times = np.append(times, time)
        cumulative_costs = np.append(cumulative_costs, cumulative_cost)


df = pd.DataFrame(
    dict(
        failure_mode=failure_modes,
        task=tasks,
        time=times,
        cost=costs,
        cumulative_cost=cumulative_costs,
    )
)

22.4 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
times = np.linspace(0,100,101, dtype=int)
n=101

time = np.array(df['time'].values.tolist())
cost = np.array(df['cost'].values.tolist())
fmt = np.repeat(df[['failure_mode', 'task']].to_numpy(),n, axis=0)
pd.DataFrame(np.column_stack((a, vals.ravel())), columns=df.columns)

NameError: name 'a' is not defined

In [6]:
profile = comp.fm['random'].expected_cost()
profile['risk'] = comp.fm['random'].expected_risk()

In [115]:
%timeit comp.fm['slow_aging'].expected_cost_df()

60.8 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [116]:
%timeit comp.fm['slow_aging'].expected_risk_cost_df()

32.1 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
rc = comp.fm['slow_aging'].expected_risk_cost()

In [18]:
new_index = pd.Index(np.arange(0, 200, 1), name="time")

t = rc['inspection']['time']
c = rc['inspection']['cost']

In [28]:
tc = dict(task=[], time=[], cost=[])

for k, v in rc.items():
    tc['task'] = np.append(tc['task'], np.full(len(v['time']), k))
    for m in ['time', 'cost']:
        time = tc[m]
        
        

        tc[m] = np.append(tc[m], v[m])

array([ 0. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. , 50. ,
       50. , 50. , 50. , 50. , 50. , 50. , 50. , 49.5])

In [9]:
df = comp.fm['random'].expected_risk_cost_df()
df.index.name = 'task'
df = df.reset_index()

In [10]:
df = comp.fm['slow_aging'].expected_risk_cost_df()

In [12]:

px.line(
    df.reset_index(),
    x="time",
    y="cumulative_cost",
    color="task",
    line_group =
)

In [63]:
df

,time,cost,cumulative_cost,1
inspection,5,50,NaN,NaN
inspection,10,50,NaN,NaN
inspection,15,50,NaN,NaN
inspection,20,50,NaN,NaN
inspection,25,50,NaN,NaN
...,...,...,...,...
on_condition_repair,200,1,NaN,NaN
cm,135,50,NaN,NaN
cm,180,50,NaN,NaN
risk,132,50000,NaN,NaN


In [13]:
comp.fm['random'].expected_risk_cost_df()

,time,cost,cumulative_cost
task,,,
inspection,0,0,0
inspection,5,50,50
inspection,10,50,100
inspection,15,50,150
inspection,20,50,200
...,...,...,...
on_condition_repair,200,2,37
cm,0,0,0
cm,60,50,50


In [22]:
%timeit profile = comp.fm['random'].expected_risk()

277 µs ± 96.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
%timeit profile = comp.fm['random'].expected_cost()

566 ms ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit profile = comp.fm['random'].tasks['inspection'].expected_costs()

336 ms ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
task = comp.fm['random'].tasks['inspection']

In [29]:

%timeit {t:task.t_completion.count(t) * task.cost / 1 for t in task.t_completion}

311 ms ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


{'inspection': {5: 50.0,
  10: 50.0,
  15: 50.0,
  20: 50.0,
  25: 50.0,
  30: 50.0,
  35: 50.0,
  40: 50.0,
  45: 50.0,
  50: 50.0,
  55: 50.0,
  60: 50.0,
  65: 50.0,
  70: 50.0,
  75: 50.0,
  80: 50.0,
  85: 50.0,
  90: 50.0,
  95: 50.0,
  100: 50.0,
  105: 50.0,
  110: 50.0,
  115: 50.0,
  120: 50.0,
  125: 50.0,
  130: 50.0,
  135: 50.0,
  140: 50.0,
  145: 50.0,
  150: 50.0,
  155: 50.0,
  160: 50.0,
  165: 50.0,
  170: 50.0,
  175: 50.0,
  180: 50.0,
  185: 50.0,
  190: 50.0,
  195: 50.0,
  200: 50.0},
 'on_condition_repair': {180: 3.0,
  95: 1.0,
  50: 3.0,
  60: 1.0,
  145: 1.0,
  195: 1.0,
  185: 1.0,
  160: 1.0,
  90: 2.0,
  170: 1.0,
  5: 2.0,
  125: 1.0,
  35: 1.0,
  30: 2.0,
  110: 2.0,
  15: 2.0,
  190: 1.0,
  65: 1.0,
  80: 2.0,
  55: 2.0,
  155: 1.0,
  10: 1.0,
  70: 1.0,
  120: 1.0,
  20: 1.0,
  45: 1.0},
 'cm': {},
 'risk': {}}